# Experiment Main

In [1]:
import pandas as pd
import numpy as np
import itertools
from tqdm.notebook import tqdm
import pickle

import torch
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.utils.data.sampler as sampler
import torch.nn as nn
import torch.nn.functional as F

from matplotlib import pyplot as plt
from collections import defaultdict
import warnings
from sklearn.preprocessing import LabelEncoder
import random
from datetime import datetime
import pytz
import line_profiler

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Fix Random Seeds

In [2]:
def same_seeds(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
  np.random.seed(seed)  
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

same_seeds(0)

## Prepare Data

In [3]:
CONTEXT_REPS = pd.read_pickle('../../data/w_final_context.pkl')
STREAM_ITEM_DICT = pd.read_pickle('../../data/stream_item_dict.pkl')
BERT_BY_IDX_DF = pd.read_pickle('../../data/bert_by_idx_pca.pkl')
BOUGHT_DICT = pd.read_pickle('../../data/bought_dict.pkl')
USER_ALL_STREAM_INIT = CONTEXT_REPS.describe().loc['50%']

In [4]:
CONTEXT_REPS.shape, len(STREAM_ITEM_DICT), BERT_BY_IDX_DF.shape, len(BOUGHT_DICT)

((1397141, 219), 7701, (162189, 160), 79207)

In [5]:
USER_LIST = CONTEXT_REPS.index.get_level_values('asid').unique().tolist()

In [6]:
LB_ITEMS = ['item_id'] + [f'i{x}' for x in range(160)]
INPUT_DF_COL__USR = CONTEXT_REPS.columns.to_list()
INPUT_DF_COL = INPUT_DF_COL__USR + LB_ITEMS

'''
METHOD FOR BOTH EXP_REPLAY & DQN
Convert state format to model input format
'''
def get_input_tensor(input_state, current_stream, with_tensor=False):
  # Get item feats
  # STREAM_ITEM_DICT: 要拿到對的 STREAM!!!
  item_list = STREAM_ITEM_DICT[current_stream]
  item_feat = BERT_BY_IDX_DF.loc[item_list].reset_index().rename(columns={'index': 'item_id'})

  # Fill in other context
  stream_item_feat = pd.DataFrame([input_state]*len(item_list)).reset_index(drop=True)
  
  # Merge with items
  stream_item_feat = stream_item_feat.merge(item_feat, left_index=True, right_index=True).astype('float32')
  
  # Convert to tensor
  if with_tensor: 
    stream_item_feat_tensor = df_to_tensor(stream_item_feat)
    return stream_item_feat_tensor, stream_item_feat
  else:
    return stream_item_feat

In [7]:
'''
METHOD FOR BOTH EXP_REPLAY & DQN

Generate series: whether elements in A existed in list B
A, B: List
return: pd.Series
example:
  A: [1, 2, 4, 5]
  B: [1, 2, 3, 4, 5, 6, 7]
  return: Series([1, 1, 0, 1, 1, 0, 0], index=[1, 2, 3, 4, 5, 6, 7])
'''
def gen_exist_series(A, B):
  return [int(item in A) for item in B]

In [8]:
def df_to_tensor(input_df):
  return torch.tensor(input_df.values).to(DEVICE).float()

---

## Replay

In [9]:
class ReplayBuffer:
  def __init__(self, max_memory=100000, discount=.9, model_output_shape=1):
    """
    Setup
    max_memory: the maximum number of experiences we want to store
    memory: a list of experiences
    discount: the discount factor for future experience
    In the memory the information whether the game ended at the state is stored seperately in a nested array
    [...
    [experience, game_over]
    [experience, game_over]
    ...]
    """
    self.max_memory = max_memory
    self.memory = list()
    self.discount = discount
    self.model_output_shape = model_output_shape

  def remember(self, states, game_over):
    # Save a state to memory
    self.memory.append([states, game_over])
    # We don't want to store infinite memories, so if we have too many, we just delete the oldest one
    if len(self.memory) > self.max_memory:
      del self.memory[0]

  def get_batch(self, eval_net, target_net, structure, batch_size=10):
    # How many experiences do we have?
    len_memory = len(self.memory)

    # Calculate the number of actions that can possibly be taken in the game.
    # Actions: 0 = not recommend, 1 = recommend
    num_actions = self.model_output_shape

    # Dimensions of our observed states, ie, the input to our model.
    # Memory:  [
    #   [ [ [stream, next_stream], [...state], action, reward, next_state_idx], game_over],
    #   [ [ [stream, next_stream], [...state], action, reward, nexr_state_idx], game_over],
    #   ...
    # ]
    env_dim = len(INPUT_DF_COL)

    inputs = pd.DataFrame()
    targets = torch.tensor([], dtype=torch.float32).to(DEVICE)
    
    
    # We draw states to learn from randomly
    for i, idx in enumerate(np.random.randint(0, len_memory, size=min(len_memory, batch_size))):  
      # Here we load one transition <s, a, r, s'> from memory
      streams, state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
      current_stream, next_stream = streams
      game_over = self.memory[idx][1]

      '''
      修改倒入 state 的方式 input = (state - item) + item_feat
      拆掉 model_predict 成 function

      here should be state_t * all_items
      '''
      state_tensor, state_t = get_input_tensor(state_t, current_stream, with_tensor=True)
      # puts state into input
      inputs = pd.concat([inputs, state_t], axis=0)

      reward_t = df_to_tensor(reward_t).view(len(reward_t), 1)

      '''
      每個 actions 都會被 predict 一個成績/reward
      '''
      # if the game ended, the reward is the final reward
      if game_over:  # if game_over is True
        current_target = reward_t
      else:
        state_tp1, _ = get_input_tensor(state_tp1, next_stream, with_tensor=True)
        if structure == 'vanilla':
          with torch.no_grad():
            Q_sa = torch.max(eval_net(state_tp1))
        elif structure == 'target':
          with torch.no_grad():
            Q_sa = torch.max(target_net(state_tp1))
        elif structure == 'double':
          with torch.no_grad():
            _, selected_actions = eval_net(state_tp1).max(dim=0, keepdim=True)
            Q_sa = target_net(state_tp1).gather(dim=0, index=selected_actions)
          
        # r + gamma * max Q(s',a')
        # current_target = reward_t + self.discount * Q_sa
        current_target = reward_t.add(Q_sa, alpha=self.discount)

      targets = torch.cat((targets, current_target), 0)
    return inputs, targets

## Epsilon

In [10]:
from abc import ABC, abstractmethod
import math

class Epsilon(ABC):
  @abstractmethod
  def clear(self):
    pass
  
  @abstractmethod
  def get_epsilon(self, key):
    pass
  
  @abstractmethod
  def update_at_step(self, key, data, delta):
    pass
  
  @abstractmethod
  def update_at_epoch(self, data):
    pass
  
  # @abstractmethod
  # def update_at_epsisode():
  #   pass


class Decay(Epsilon):
  # Ref: Decay(0.5, 0.85)
  '''
  Epsilon Decay EE method with update/decay at epoch
  '''
  def __init__(self, initial, epoch_decay, step_decay=1.0):
    self.initial = initial
    self.epoch_decay, self.step_decay = epoch_decay, step_decay
    self.epsilon = self.initial
    
  def clear(self):
    self.epsilon = self.initial # should be 4 for origin setting
    
  def get_epsilon(self, key):
    return self.epsilon
  
  def update_at_step(self, key, data, delta):
    # origin setting
    pass
    # exponentially
    # self.epsilon *= self.step_decay
    
  def update_at_epoch(self, data):
    # origin settings
    epoch = data
    self.epsilon = 4 / ((epoch + 1) ** (1 / 2))
    # exponentially
    # self.epsilon *= self.epoch_decay


class VDBE(Epsilon):
  # VDBE(0.5, 0.01)
  def __init__(self, initial, sigma):
    self.initial = initial
    self.sigma = sigma

  def clear(self):
    self.epsilon = defaultdict(lambda: self.initial)

  def get_epsilon(self, key):
    return self.epsilon[key]
  
  def update_at_step(self, key, data, delta):
    td_error = data
    coeff = math.exp(-abs(td_error) / self.sigma)
    f = (1.0 - coeff) / (1.0 + coeff)
    self.epsilon[key] = delta * f + (1.0 - delta) * self.epsilon[key]
  
  def update_at_epoch(self, data):
    pass

## DQN

In [11]:
class DQN(object):
  def __init__(self, structure, exp_replay, epsilon, num_episode, epochs, batch_size, lr, switch_param_threshold, single_reward):
    self.eval_net = Net()
    self.target_net = Net() if not structure == 'vanilla' else None
    self.structure = structure
    self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=lr)
    self.loss_fn = nn.MSELoss()
    self.exp_replay = exp_replay
    self.epsilon = epsilon
    self.num_episode = num_episode
    self.epochs = epochs
    self.batch_size = batch_size
    self.switch_param_threshold = switch_param_threshold
    self.single_reward = single_reward
    self.user_all_stream_init = USER_ALL_STREAM_INIT
    self.win_list = []
    self.score_list = []
    self.loss_list = []
    self.learn_step_counter = 0

  # Environment Methods
  def __episodes(self):
    # return USER_LIST[:self.num_episode]
    return np.random.choice(USER_LIST, self.num_episode, replace=False)
  
  def __user_episode_context(self):
    self.user_all_streams = CONTEXT_REPS.xs(self.asid, level="asid")
    self.stream_list = self.user_all_streams.index
    self.final_stream = max(self.stream_list)
  
  def __full_state(self, i):
    '''
    retrieve full state -> should be exported to pickle
    '''
    if (i - 1) == -1:
      user_part = self.user_all_stream_init.copy()
      user_part.name = self.stream_list[i]
    else:
      user_part = self.user_all_streams.loc[self.stream_list[(i - 1)]]
    return user_part
    
  def _print_status(self):
    if self.rec_cnt % 100 == 0:
      self.win_list.append(self.win_cnt)
      self.score_list.append(self.score)
      self.loss_list.append(self.loss)
      print(f'[{len(self.win_list)}] win_cnt: {self.win_cnt} | score: {self.score} | asid: {USER_LIST.index(self.asid)} | loss: {self.loss:.2f} | explore: {self.explore} | exploit: {self.exploit}')
      self.win_cnt = 0
      self.score = 0
      self.loss = 0
      self.explore = 0
      self.exploit = 0
      
  def reward(self):
    '''
    Comparison function for reward, 考慮「所有」歷史購買紀錄
    '''
    real_bought_ids = BOUGHT_DICT[self.asid]
    real_bought_ids_series = gen_exist_series(real_bought_ids, self.stream_items)
    
    reward_list = [a & b for a, b in zip(real_bought_ids_series, self.action_ids)]

    # Reward Count 
    self.rec_cnt += 1
    if sum(reward_list) > 0:
      self.win_cnt += 1
      self.score += sum(reward_list)
    self._print_status()
    
    if self.single_reward:
      return pd.Series(reward_list, index=self.stream_items)
    else:
      return pd.Series(list(map(lambda x: x * sum(reward_list), reward_list)), index=self.stream_items)

  # Agent Methods
  def __choose_actions(self):
    if np.random.rand() <= self.epsilon.get_epsilon(self.asid):
    # if len(self.exp_replay.memory) < 1:
      # Explore by randomly select 10/n items from candidate_items
      # Get all items from the stream
      self.explore += 1
      selected_actions = random.sample(self.stream_items, 10) if len(self.stream_items) > 10 else self.stream_items
    else:
      # Exploit by choosing action from the model's prediction
      self.exploit += 1
      selected_actions = self.__agent_predict()
    x = pd.Series(0, index=self.stream_items)
    x.loc[selected_actions] = 1
    return x
    
  def q_value(self): 
    if type(self.epsilon) == Decay: return 0
    with torch.no_grad():
      predicts = self.eval_net(self.full_input).flatten()    
    actions_idx = np.where(self.action_ids.values == 1)[0]
    q_val = predicts[actions_idx].mean()
    return q_val

  def __agent_predict(self):
    with torch.no_grad():
      predicts = self.eval_net(self.full_input).flatten()
    if len(predicts) > 10:
      top10_idx = torch.topk(predicts, 10).indices.cpu()
      actions = self.candidate_actions.iloc[top10_idx]['item_id'].values
    else:
      actions = self.candidate_actions['item_id'].values
    return actions

  def __train_agent_batch(self, inputs, targets):
    self.optimizer.zero_grad()
    outputs = self.eval_net(inputs)
    loss = self.loss_fn(outputs, targets)
    # Add CL Regularization Term
    loss.backward()
    self.optimizer.step()
    return loss.item()

  # MAIN TRAIN
  def train(self):
    self.eval_net.to(DEVICE)
    if self.target_net:
      self.target_net.to(DEVICE)
    self.eval_net.train(True)
    self.epsilon.clear()
    self.loss = 0.
    self.rec_cnt = 0
    self.win_cnt = 0
    self.explore = 0
    self.exploit = 0
    self.score = 0

    # ------------------- Episode (User) -------------------------------
    for asid in tqdm(USER_LIST):
      self.asid = asid
      self.__user_episode_context()

      # ----------------- Runs (User x All_Stream) ---------------------
      for i, stream in enumerate(self.stream_list):
        game_over = stream == self.final_stream
        self.current_stream = stream
        self.current_state = self.__full_state(i)
        self.stream_items = STREAM_ITEM_DICT[self.current_stream]
        self.full_input, self.candidate_actions = get_input_tensor(self.current_state, self.current_stream, with_tensor=True)

        # --------------- Explore/Exploit Section ----------------------
        self.action_ids = self.__choose_actions()

        # --------------- Get next state & info to store ---------------
        reward = self.reward()
        next_state = self.__full_state(i+1) if not game_over else []
        next_stream = 0 if (i + 1) == len(self.stream_list) else self.stream_list[i + 1]
        self.exp_replay.remember([[stream, next_stream], self.current_state, self.action_ids, reward, next_state], game_over)
        self.learn_step_counter += 1
        if self.target_net and (self.learn_step_counter % self.switch_param_threshold == 0):
          self.target_net.load_state_dict(self.eval_net.state_dict())


        # --------------- Load batch of experiences --------------------
        inputs, targets = self.exp_replay.get_batch(self.eval_net, self.target_net, self.structure, batch_size=self.batch_size)
        inputs = df_to_tensor(inputs)
        # store pre-training value for td_error
        old_Q = self.q_value()
        batch_loss = self.__train_agent_batch(inputs, targets)
        # store post-training value for td_error
        new_Q = self.q_value()
        self.loss += batch_loss

        # --------------- Update with TD error -------------------------
        self.epsilon.update_at_step(self.asid, (new_Q - old_Q), len(self.stream_items))

## Main Method

In [12]:
pd.set_option('mode.chained_assignment', None)

# parameters
MAX_MEMORY = 1000  # Maximum number of experiences we are storing
BATCH_SIZE = 2  # Number of experiences we use for training per batch
EPOCH = range(100)
TOTAL_ACTIONS = 1 # probability of ordering
NUM_EPISODE = 100
HIDDEN_SIZE = 512
LR = 1.0e-4
SWITCH_PARAM_THRESHOLD = 100

warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(380, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 128)
    self.fc4 = nn.Linear(128, 64)
    self.fc5 = nn.Linear(64, 1)
    self.relu = nn.ReLU()
    self.tanh = nn.Tanh()

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.relu(x)
    x = self.fc4(x)
    x = self.tanh(x)
    x = self.fc5(x)
    return x

## Exp: Reward Setting

In [ ]:
'''
EPS_baseline_rsum_x
'''

exp_replay2 = ReplayBuffer(max_memory=MAX_MEMORY)
epsilon2 = Decay(0.5, 0.85)
EPS_baseline_rsum_x = DQN('vanilla', exp_replay2, epsilon2, NUM_EPISODE, EPOCH, BATCH_SIZE, LR, SWITCH_PARAM_THRESHOLD, single_reward=False)
EPS_baseline_rsum_x.train()

  0%|          | 0/79207 [00:00<?, ?it/s]

[1] win_cnt: 47 | score: 78 | asid: 5 | loss: 9.00 | explore: 50 | exploit: 50
[2] win_cnt: 76 | score: 162 | asid: 5 | loss: 10.94 | explore: 52 | exploit: 48
[3] win_cnt: 57 | score: 88 | asid: 10 | loss: 14.49 | explore: 50 | exploit: 50
[4] win_cnt: 55 | score: 78 | asid: 20 | loss: 5.26 | explore: 40 | exploit: 60
[5] win_cnt: 60 | score: 85 | asid: 24 | loss: 8.54 | explore: 46 | exploit: 54
[6] win_cnt: 58 | score: 98 | asid: 31 | loss: 16.09 | explore: 49 | exploit: 51
[7] win_cnt: 46 | score: 91 | asid: 35 | loss: 11.65 | explore: 46 | exploit: 54
[8] win_cnt: 45 | score: 61 | asid: 44 | loss: 10.63 | explore: 53 | exploit: 47
[9] win_cnt: 68 | score: 87 | asid: 46 | loss: 5.42 | explore: 49 | exploit: 51
[10] win_cnt: 57 | score: 81 | asid: 54 | loss: 6.38 | explore: 50 | exploit: 50
[11] win_cnt: 50 | score: 63 | asid: 58 | loss: 6.19 | explore: 50 | exploit: 50
[12] win_cnt: 56 | score: 92 | asid: 62 | loss: 9.91 | explore: 47 | exploit: 53
[13] win_cnt: 43 | score: 57 | as

In [4]:
# plt.plot(EPS_baseline_rsum_x.score_list, label='VDBE hit@10')
# plt.plot(baseline100, label='baseline hit@10')
plt.plot(pd.Series(EPS_baseline_rsum_x.score_list).rolling(1500).mean(), label='VDBE hit@10 ma 30')
# plt.plot(pd.Series(baseline100).rolling(30).mean(), label='Baseline hit@10 ma 30')
plt.xlabel('Epoch')
plt.ylabel('Hit Ratio')
plt.title('Hit@10 for BERT-baseline')
# plt.ylim([0.4, 0.6])
plt.legend()

NameError: name 'EPS_baseline_rsum_x' is not defined

In [30]:
with open('../Models/EPS_baseline_rsum_x_noepoch.pkl', 'wb') as file_pi:
  pickle.dump(EPS_baseline_rsum_x, file_pi, pickle.HIGHEST_PROTOCOL)

In [39]:
dqn.c_hist[-1]/sum(dqn.rec_list)

0.5406706451835664

In [17]:
'''
請不要關掉這ㄍ分頁 乾蝦哈咪搭
'''

'\n請不要關掉這ㄍ分頁 乾蝦哈咪搭\n'

In [24]:
with open('../Models/vdbe_100_target.pkl', 'wb') as file_pi:
  pickle.dump(dqn, file_pi, pickle.HIGHEST_PROTOCOL)

In [193]:
%load_ext line_profiler
# %reload_ext line_profiler

In [107]:
%%time
exp_replay2 = ReplayBuffer(max_memory=MAX_MEMORY)
epsilon2 = VDBE(0.5, 0.01)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 10.3 µs


In [108]:
%%time
dqn2 = DQN(exp_replay2, epsilon2, 5, range(2), BATCH_SIZE, LR, SWITCH_PARAM_THRESHOLD)

CPU times: user 8.22 s, sys: 172 ms, total: 8.4 s
Wall time: 8.39 s


### Start Chasing

In [109]:
%lprun -f dqn2.train dqn2.train()

Epoch 0 started.   Time: 20:42:38


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0/2 | Loss 29.661170213483274 | Epoch Hit Rate 0.5357142857142857 |               Cumulative Hit Rate 0.5357142857142857 | Explore 35 | Exploit 49 | Score 73
Epoch 1 started.   Time: 20:42:52


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1/2 | Loss 21.957408252041205 | Epoch Hit Rate 0.5808383233532934 |               Cumulative Hit Rate 0.5657370517928287 | Explore 126 | Exploit 125 | Score 160


Timer unit: 1e-09 s

Total time: 43.5841 s
File: <ipython-input-106-35d26399f5ef>
Function: train at line 102

Line #      Hits         Time  Per Hit   % Time  Line Contents
   102                                             def train(self):
   103         1   14851547.0 14851547.0      0.0      self.eval_net.to(DEVICE)
   104         1     470070.0 470070.0      0.0      self.target_net.to(DEVICE)
   105         1        630.0    630.0      0.0      self.c_win_cnt = 0
   106         1      41511.0  41511.0      0.0      self.eval_net.train(True)
   107         1       3745.0   3745.0      0.0      self.epsilon.clear()
   108         1        403.0    403.0      0.0      self.explore = 0
   109         1        293.0    293.0      0.0      self.exploit = 0
   110                                           
   111         2       1402.0    701.0      0.0      for e in self.epochs:
   112         2        774.0    387.0      0.0        self.rec_cnt = 0
   113         2        580.0    290

### old profiling

In [118]:
%lprun -f exp_replay2.get_batch exp_replay2.get_batch(dqn2.eval_net, dqn2.target_net, batch_size=2)

Timer unit: 1e-09 s

Total time: 0.0906188 s
File: <ipython-input-103-790fe20007f1>
Function: get_batch at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                             def get_batch(self, eval_net, target_net, batch_size=10):
    27                                               # How many experiences do we have?
    28         1       1053.0   1053.0      0.0      len_memory = len(self.memory)
    29                                           
    30                                               # Calculate the number of actions that can possibly be taken in the game.
    31                                               # Actions: 0 = not recommend, 1 = recommend
    32         1        399.0    399.0      0.0      num_actions = self.model_output_shape
    33                                           
    34                                               # Dimensions of our observed states, ie, the input to our model.
    35  

In [119]:
%lprun -f get_input_tensor get_input_tensor(CONTEXT_REPS.iloc[0], CONTEXT_REPS.iloc[0].name[1], with_tensor=True)

Timer unit: 1e-09 s

Total time: 0.0201811 s
File: <ipython-input-102-0d98a1c59d51>
Function: get_input_tensor at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def get_input_tensor(input_state, current_stream, with_tensor=False):
    10                                             # Get item feats
    11                                             # STREAM_ITEM_DICT: 要拿到對的 STREAM!!!
    12         1       2116.0   2116.0      0.0    item_list = STREAM_ITEM_DICT[current_stream]
    13         1    2208028.0 2208028.0     10.9    item_feat = BERT_BY_IDX_DF.loc[item_list].reset_index().rename(columns={'index': 'item_id'})
    14                                           
    15                                             # Fill in other context
    16         1   16447950.0 16447950.0     81.5    stream_item_feat = pd.DataFrame([input_state]*len(item_list)).reset_index(drop=True)
    17                               

### Cleansing
Original Version:
```python
'''
METHOD FOR BOTH EXP_REPLAY & DQN

Convert state format to model input format
'''
def get_input(input_state, current_stream):
  # Get item feats
  # STREAM_ITEM_DICT: 要拿到對的 STREAM!!!
  item_list = STREAM_ITEM_DICT[current_stream]
  item_feat = BERT_BY_IDX_DF.loc[item_list]

  # Create new df
  stream_item_feat = pd.DataFrame(columns=INPUT_DF_COL)

  # Fill in other context
  stream_item_feat = stream_item_feat.append([input_state]*len(item_list),ignore_index=True)
  
  # stream_item_feat
  stream_item_feat[LB_ITEMS] = item_feat.reset_index()
  
  return stream_item_feat.astype('float32')
```

In [25]:
%load_ext heat

In [31]:
'''
METHOD FOR BOTH EXP_REPLAY & DQN
Convert state format to model input format

LB_ITEMS = ['item_id'] + [f'i{x}' for x in range(160)]
INPUT_DF_COL__USR = CONTEXT_REPS.columns.to_list()
INPUT_DF_COL = INPUT_DF_COL__USR + LB_ITEMS
'''
input_state = CONTEXT_REPS.iloc[0]
current_stream = CONTEXT_REPS.iloc[0].name[1]
def new_get_input(input_state, current_stream):
  # Get item feats
  # STREAM_ITEM_DICT: 要拿到對的 STREAM!!!
  item_list = STREAM_ITEM_DICT[current_stream]
  item_feat = BERT_BY_IDX_DF.loc[item_list]

  for idx in input_state.index: item_feat[idx] = input_state[idx]
  item_feat_ri = item_feat.reset_index().rename(columns={'index':'item_id'})
  
  return item_feat_ri

In [23]:
%lprun -f new_get_input new_get_input(CONTEXT_REPS.iloc[0], CONTEXT_REPS.iloc[0].name[1])

Timer unit: 1e-09 s

Total time: 0.107122 s
File: <ipython-input-22-07c49d6a115a>
Function: new_get_input at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def new_get_input(input_state, current_stream):
    12                                             # Get item feats
    13                                             # STREAM_ITEM_DICT: 要拿到對的 STREAM!!!
    14         1       2309.0   2309.0      0.0    item_list = STREAM_ITEM_DICT[current_stream]
    15         1     396509.0 396509.0      0.4    item_feat = BERT_BY_IDX_DF.loc[item_list]
    16                                           
    17       219  105267435.0 480673.2     98.3    for idx in input_state.index: item_feat[idx] = input_state[idx]
    18         1    1455838.0 1455838.0      1.4    item_feat_ri = item_feat.reset_index().rename(columns={'index':'item_id'})
    19                                             
    20         1        133.0    1

In [39]:
def get_input_v2(input_state, current_stream):
  # Get item feats
  item_list = STREAM_ITEM_DICT[current_stream]
  item_feat = BERT_BY_IDX_DF.loc[item_list].reset_index().rename(columns={'index': 'item_id'})

  # Fill in other context
  stream_item_feat = pd.DataFrame([input_state]*len(item_list)).reset_index(drop=True)
  
  # Merge with items
  stream_item_feat = stream_item_feat.merge(item_feat, left_index=True, right_index=True).astype('float32')
  return stream_item_feat

In [85]:
%lprun -f get_input_v2 get_input_v2(CONTEXT_REPS.iloc[0], CONTEXT_REPS.iloc[0].name[1])

Timer unit: 1e-09 s

Total time: 0.0176343 s
File: <ipython-input-84-4d017c41242b>
Function: get_input_v2 at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def get_input_v2(input_state, current_stream):
    10                                             # Get item feats
    11                                             # STREAM_ITEM_DICT: 要拿到對的 STREAM!!!
    12         1       8785.0   8785.0      0.0    item_list = STREAM_ITEM_DICT[current_stream]
    13         1    5560119.0 5560119.0     31.5    item_feat = BERT_BY_IDX_DF.loc[item_list].reset_index().rename(columns={'index': 'item_id'})
    14                                           
    15                                             # Fill in other context
    16         1   10882756.0 10882756.0     61.7    stream_item_feat = pd.DataFrame([input_state]*len(item_list)).reset_index(drop=True)
    17                                             
    18       

### Chasing 2

In [88]:
%%time
baseline_model3 = Baseline()

CPU times: user 115 µs, sys: 2.91 ms, total: 3.02 ms
Wall time: 2.34 ms


In [89]:
%%time
exp_replay3 = ReplayBuffer(max_memory=MAX_MEMORY)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs


In [90]:
%%time
dqn3 = DQN(baseline_model3, exp_replay3, 5, EPOCH, BATCH_SIZE, LR)

CPU times: user 8.32 s, sys: 177 ms, total: 8.5 s
Wall time: 8.49 s


### Start Chasing

In [91]:
%lprun -f dqn3.train dqn3.train()

Epoch 0 started.   Time: 15:23:38


  0%|          | 0/5 [00:00<?, ?it/s]

Timer unit: 1e-09 s

Total time: 3.32292 s
File: <ipython-input-87-1e430f3ba6d5>
Function: train at line 93

Line #      Hits         Time  Per Hit   % Time  Line Contents
    93                                             def train(self):
    94         1   16194974.0 16194974.0      0.5      self.model.to(DEVICE)
    95         1        950.0    950.0      0.0      self.c_win_cnt = 0
    96         1      40919.0  40919.0      0.0      self.model.train(True)
    97                                           
    98         1        738.0    738.0      0.0      for e in self.epochs:
    99         1        361.0    361.0      0.0        self.rec_cnt = 0
   100         1        335.0    335.0      0.0        self.win_cnt = 0
   101         1        395.0    395.0      0.0        self.loss = 0.
   102         1       1624.0   1624.0      0.0        self.epsilon = 4 / ((e + 1) ** (1 / 2))
   103                                           
   104         1      99669.0  99669.0      0.0    

Epoch: 0/100 | Loss 2.4590116441249847 | Epoch Hit Rate 0.775 | Cumulative Hit Rate 0.775 |              Time 15:23:41


In [92]:
%lprun -f exp_replay3.get_batch exp_replay3.get_batch(baseline_model3, batch_size=2)

Timer unit: 1e-09 s

Total time: 0.0848023 s
File: <ipython-input-86-f9f91d8b7747>
Function: get_batch at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                             def get_batch(self, model, batch_size=10):
    27                                               # How many experiences do we have?
    28         1       1013.0   1013.0      0.0      len_memory = len(self.memory)
    29                                           
    30                                               # Calculate the number of actions that can possibly be taken in the game.
    31                                               # Actions: 0 = not recommend, 1 = recommend
    32         1        741.0    741.0      0.0      num_actions = self.model_output_shape
    33                                           
    34                                               # Dimensions of our observed states, ie, the input to our model.
    35                  

Per hit
* exp/get_input:
  * v1: `49609136.0`
  * v2: `12412833.5`
* train/get_batch:
  * v1: `216278453.8`
  * v2: `78445226.8`

In [66]:
predicts = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
if len(predicts) > 10:
  ind = np.argpartition(predicts, -10)[-10:]
  q_val = predicts[ind].sum()
else:
  q_val = predicts.sum()
q_val

tensor(85)

In [67]:
predicts[ind]

tensor([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])